In [96]:
from dataclasses import dataclass
from typing import List, Optional, Union, Any, Callable
from collections import defaultdict
import json
import os
import math
from functools import partial
import string
from string import ascii_letters
from textwrap import dedent
import re

import numpy as np
import pandas as pd
import yaml

In [82]:
test_data = """
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
""".strip()


@dataclass
class Monkey:
    """Represent Monkey in Day 11."""

    starting_items: List[int]
    operation: Callable[[int], int]
    test_value: int  # Divisible by int.
    test_result_actions: tuple[int, int]  # false, true.


def _operation_callable_base(x: int, value: str, operation: str) -> int:
    """Use to create a partial for ``operation_callable`` in ``parse_input()``."""
    if operation == "+":
        return x + int(value)

    if operation == "*":
        if value == "old":
            return x**2
        return x * int(value)

    raise ValueError(f"Cannot parse {x}, {value}, {operation}")


def parse_input(data: str) -> dict[int, Monkey]:
    """Parse input data for Day 11."""
    # NOTE: This can be broken down to smaller functions to unit test.
    monkeys = [[row.strip() for row in line.split("\n")] for line in data.split("\n\n")]

    monkey_dict: dict[int, Monkey] = {}
    for monkey in monkeys:
        monkey_key = int(re.findall(r"Monkey (\d+)?:", monkey[0])[0])

        # Get the numeric values in "Starting items: ..." line.
        starting_items = list(map(int, re.findall(": (.*)", monkey[1])[0].split(", ")))

        # This is always + or *, so we parse and create a callable.
        operation_str = re.findall(r": new = old (.*)", monkey[2])[0].strip()
        operation_list = operation_str.split(" ")

        operation_callable = partial(
            _operation_callable_base, value=operation_list[1], operation=operation_list[0]
        )

        # Parse this out to a number as it is always "divisible by some number."
        test_value = int(re.findall(r"Test: divisible by (\d+)", monkey[3])[0])

        # (false_value, true_value), and these are ints representing the monkey thrown to.
        true_val = int(re.findall(r"If true: throw to monkey (\d+)", monkey[4])[0])
        false_val = int(re.findall(r"If false: throw to monkey (\d+)", monkey[5])[0])
        test_result_actions = (false_val, true_val)

        monkey_dict[monkey_key] = Monkey(
            starting_items=starting_items,
            operation=operation_callable,
            test_value=test_value,
            test_result_actions=test_result_actions,
        )

    return monkey_dict

In [132]:
class Round:
    """Represent a round in Day 11 of Monkeys doing things."""
    def __init__(self, monkey_data: dict[int, Monkey]):
        self.monkey_data = monkey_data
        self.monkey_num_times_inspected = {idx: 0 for idx in range(len(self.monkey_data))}
        self.current_items = [self.monkey_data[i].starting_items for i in range(len(self.monkey_data))]
        self.current_round = 1

    def do_a_round(self):
        print(f"Round {self.current_round}")
        for idx in range(len(self.monkey_data)):
            # print(f"  Monkey {idx}:")
            self.individual_monkey_round(monkey_index=idx)
        self.current_round += 1

        # Update current values for items
        self.current_items = [self.monkey_data[i].starting_items for i in range(len(self.monkey_data))]


    def individual_monkey_round(self, monkey_index: int):
        current_monkey_data = self.monkey_data[monkey_index]
        for item in current_monkey_data.starting_items:
            self.monkey_num_times_inspected[monkey_index] += 1

            output_log = []
            output_log.append(f"Monkey inspects an item with a worry level of {item}.")
            
            worry_level = current_monkey_data.operation(item)
            output_log.append(f"Worry level is now {worry_level}")
            
            worry_level =int(math.floor(worry_level / 3))
            output_log.append(f"Monkey gets bored with item. Worry level is divided by 3 to {worry_level}.")

            if worry_level % current_monkey_data.test_value == 0:
                output_log.append("Current worry level is divisible by 23.")
                self.monkey_data[current_monkey_data.test_result_actions[1]].starting_items.append(worry_level)
                output_log.append(f"Item with worry level {worry_level} is thrown to monkey {current_monkey_data.test_result_actions[1]}.")
            else:
                output_log.append("Current worry level is not divisible by 23.")
                self.monkey_data[current_monkey_data.test_result_actions[0]].starting_items.append(worry_level)
                output_log.append(f"Item with worry level {worry_level} is thrown to monkey {current_monkey_data.test_result_actions[0]}.")
            
            # print("\n".join(output_log))
            #print()
            
        # Remove the current items.
        current_monkey_data.starting_items = []


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
[[10, 12, 14, 26, 34], [245, 93, 53, 199, 115], [], []]
{0: 101, 1: 95, 2: 7, 3: 105}
